In [ ]:
#get sentences from xml file 
import os
from xml.etree import ElementTree
file_name = 'file'
full_file = os.path.join(file_name)

dom = ElementTree.parse(full_file)

sentences = dom.findall('/Review/sentences/sentence/text')
print("loaded")

In [ ]:
#import relevant models
!pip install transformers

import torch
from transformers import BertTokenizer, BertModel, BertForSequenceClassification
import pandas as pd
import numpy as np
import time
import pprint
import sys

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
print("Belangrijke libraries zijn geimporteerd.")

In [ ]:

# Load pre-trained model (weights)
#
model = BertModel.from_pretrained('bert-base-uncased',output_hidden_states = True,)                    #pretrained
#model = BertModel.from_pretrained('/Users/markrademaker/Desktop/PostBERTModel100K',output_hidden_states = True,)                    #posttrained
#model = BertModel.from_pretrained('/Users/markrademaker/Desktop/BERT_FineTuned_Final',output_hidden_states = True,)                    #posttrained
#model = BertForSequenceClassification.from_pretrained('Desktop/SavedModel',output_hidden_states = True,) #finetuned

# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()
print("BERT model is gedownload.")

In [ ]:
sentenceList = []
for s in sentences:
    sentenceList.append(s.text)

In [ ]:
start = time.time()
bert_vectors = []
sentence_counter = 0
#loops over the reviews
for sen in sentenceList:
    result = []
    tokenized_text = tokenizer.tokenize(sen)
    #change to 512 or shorter
    if len(tokenized_text)>=512:
        del tokenized_text[512:len(tokenized_text)]
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1] * len(tokenized_text)
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])
    with torch.no_grad():
        outputs = model(tokens_tensor, segments_tensors)
        hidden_states = outputs.hidden_states
    token_embeddings = torch.stack(hidden_states, dim=0)
    token_embeddings = torch.squeeze(token_embeddings, dim=1)
    token_embeddings = token_embeddings.permute(1,0,2)
    token_vecs_sum = []
    for token in token_embeddings:
        sum_vec = torch.sum(token[-4:], dim=0)
        token_vecs_sum.append(sum_vec)
    token_vecs = hidden_states[-2][0]
    sentence_embedding = torch.mean(token_vecs, dim=0)
    #get all results for one review and display
    # replace with whole vector!
    i=0
    while i < len(tokenized_text):
        veccie = [round(vec,4) for vec in token_vecs_sum[i].tolist()]
        result.append([tokenized_text[i], veccie])
        i += 1
    bert_vectors.append(result)
    del result
    sentence_counter+=1
end = time.time()
print("Tijd: ", end-start, "s")
print("Sentences zijn getokenized en omgezet naar vectoren!")

In [ ]:
#Write in correct format for HAABSA
senCount = 0
bertVecString =''
outF = open("myOutFileTEST.txt","w")
for i in bert_vectors:
    senCount+=1
    for j in i:
        bertVecString =''
        word = str(j[0])
        vector = str(j[1])
        vector = vector.replace(",", '')
        vector = vector.replace("[",'')
        vector = vector.replace(']','')
        bertVecString += word + " "
        bertVecString += vector
        outF.write(bertVecString)
        outF.write("\n")
outF.close()
print("All", senCount," sentences now in" , outF)